In [1]:
# looking at phrase boolean queries

%matplotlib inline

import sys 
import os 

nb_dir = os.getcwd()
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from plotlib.loaders import *
from plotlib.plotters import *

from phdconf import stop
from phdconf import config 

from typing import List, Dict
import math 
import copy
from collections import OrderedDict

from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

ylims=[0.67, 0.45, 0.60, 0.59, 0.38]

In [2]:
queries = load_queries(config.AUS_TOPIC_PATH)
broad, specific = load_query_types(queries)
law, fact, generic = load_query_focus_types(queries)

In [3]:
index_names = ['filtered-phrasestop']#, config.SIGIR_INDEX_NAME]
qrel_paths = [config.AUS_QREL_PATH]#, config.SIGIR_QREL_PATH]
rel_levels = [config.AUS_REL_LEVEL]#, config.SIGIR_REL_LEVEL]
display_names = ['AUS']#, 'SIGIR']

In [4]:
BASE_DIR = os.path.join(os.environ["HOME"], 'phd-generated')

In [5]:
base_df = load_1d_dfs(index_names, qrel_paths, os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, 1050, 1050, 1)[0][0]
base_qry = load_1d_dfs(index_names, qrel_paths, os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, 1050, 1050, 1, per_query=True)[0][0]

In [ ]:
base_qry_set = load_1d_dfs(index_names, qrel_paths, os.path.join(os.environ["HOME"], 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, 1050, 1050, 1, per_query=True)[0][0]

In [9]:
def res_per_qry(path:str):
    res = {}
    with open(path) as f:
        for line in f:
            parts = line.split()
            if parts[0] not in res:
                res[parts[0]] = 0
                
            res[parts[0]] += 1
            
    return res

set_res = res_per_qry(os.path.join(os.environ["HOME"], 'dirichlet_prior', 'case-topics-filtered-phrasestop-unigram_dir_mu_1050.00.run'))

In [13]:
np.asarray(list(set_res.values())).mean()

35300.484210526316

In [14]:
np.asarray(list(res_per_qry(os.path.join(BASE_DIR, 'phrase-all-or.run')).values())).mean()

21422.54651162791

In [21]:
small_queries = [7, 24, 57, 70, 76, 85, 96]

In [9]:
runs = ['phrase-all-or.run', 'phrase-all-and.run', 'phrase-or.run', 'phrase-and.run']
names = ['all-or', 'all-and', 'or', 'and']

In [33]:
all_qry = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, BASE_DIR, [runs[0]], per_query=True)

In [34]:
diff = all_qry[0]-base_qry
metrics = copy.copy(config.METRIC_NAMES)
del metrics['unjudged@20']
fig = diff[metrics].rename(metrics, axis='columns').plot.box(fontsize=15, boxprops=dict(linestyle='-', linewidth=2), color=dict(boxes='black', whiskers='black', caps='r'), medianprops=dict(linestyle='-', linewidth=2, color='b'), figsize=(16, 4)).axhline(y=0, xmin=0.0, xmax=1.0, linestyle='--', linewidth=1.0, color='grey')

<Figure size 1152x288 with 1 Axes>

In [35]:
fig.get_figure().savefig('figures/ausnl-all-or-phrase-qry-comp.pdf')

In [44]:
diff2 = all_qry[0]-base_qry
diff2 = diff2[diff2.index.isin(small_queries)]
fig = diff2[metrics].rename(metrics, axis='columns').plot.box(fontsize=15, boxprops=dict(linestyle='-', linewidth=2), color=dict(boxes='black', whiskers='black', caps='r'), medianprops=dict(linestyle='-', linewidth=2, color='b'), figsize=(16, 4)).axhline(y=0, xmin=0.0, xmax=1.0, linestyle='--', linewidth=1.0, color='grey')

<Figure size 1152x288 with 1 Axes>

In [58]:
diff2[metrics]

,recip_rank,err@20,recall_20,recall_100,ndcg,rbp@0.80
7,0.0,0.00234,0.0000,-0.1111,-0.0325,0.0619
24,0.0,-0.38789,-0.0588,0.0000,-0.0868,-0.0318
57,0.0,0.00633,-0.1591,-0.6591,-0.4117,-0.0904
70,0.0,0.24041,-0.2727,-0.6364,-0.4094,-0.2127
76,0.0,0.03404,0.0834,0.0000,0.0424,0.1109
85,0.5,-0.11005,-0.2553,-0.6808,-0.5177,-0.2521
96,0.0,-0.00623,-0.0625,-0.1250,-0.0966,0.0511


In [52]:
base_qry[base_qry.index.isin(small_queries)][metrics]

,recip_rank,err@20,recall_20,recall_100,ndcg,rbp@0.80
24,1.0,0.91679,0.7647,0.8235,0.7857,0.8159
57,1.0,0.51116,0.3182,0.8182,0.6739,0.7233
7,1.0,0.52120,0.7778,0.9444,0.7235,0.7620
70,1.0,0.66743,0.4545,0.8182,0.7367,0.6699
76,1.0,0.23080,0.5833,1.0000,0.5962,0.4891
85,0.5,0.31708,0.2979,0.7234,0.5944,0.5545
96,1.0,0.89360,0.2500,0.3125,0.4093,0.4369


In [56]:
write_table('tables/ausnl-phrase-qry-diff', diff2[metrics].rename(columns=metrics).round(4).to_latex(escape=False))

In [45]:
fig.get_figure().savefig('figures/ausnl-all-or-phrase-qry-comp-small-queries.pdf')

In [ ]:
dfs = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, BASE_DIR, runs)

In [8]:
df = pd.DataFrame(dfs+[base_df], index=names+['base'])

In [9]:
df[df.index.isin(config.METRIC_NAMES)]

,recip_rank,unjudged@20,recall_20,recall_100,ndcg,set_P,set_recall,set_F_.5,rbp@0.10,rbp-res@0.10,rbp@0.50,rbp-res@0.50,rbp@0.80,rbp-res@0.80,err@20


In [10]:
print(df[df.index.isin(config.METRIC_NAMES)].rename(index=config.METRIC_NAMES).round(4).to_latex())

\begin{tabular}{lrrrrrrrrrrrrrrr}
\toprule
Empty DataFrame
Columns: Index(['recip\_rank', 'unjudged@20', 'recall\_20', 'recall\_100', 'ndcg', 'set\_P',
       'set\_recall', 'set\_F\_.5', 'rbp@0.10', 'rbp-res@0.10', 'rbp@0.50',
       'rbp-res@0.50', 'rbp@0.80', 'rbp-res@0.80', 'err@20'],
      dtype='object')
Index: Index([], dtype='object') \\
\bottomrule
\end{tabular}



In [11]:
df

,recip_rank,unjudged@20,recall_20,recall_100,ndcg,set_P,set_recall,set_F_.5,rbp@0.10,rbp-res@0.10,rbp@0.50,rbp-res@0.50,rbp@0.80,rbp-res@0.80,err@20
all-or,0.594071,1.046512,0.467912,0.679543,0.490156,0.024016,0.880097,0.028233,0.478608,0.000106,0.418827,0.002381,0.320749,0.018309,0.382911
all-and,0.553795,1.605263,0.437292,0.602312,0.433595,0.041255,0.761229,0.037187,0.438843,0.028026,0.380308,0.044076,0.276974,0.089614,0.359284
or,0.570085,3.988372,0.416064,0.563498,0.431833,0.032801,0.646590,0.040559,0.477524,0.047829,0.412557,0.063060,0.311784,0.128351,0.384968
and,0.445387,5.075472,0.221138,0.258772,0.235642,0.101066,0.273513,0.077287,0.386143,0.141343,0.322034,0.221211,0.219274,0.391240,0.270142
base,0.568606,0.000000,0.459971,0.721831,0.461328,0.079579,0.721831,0.107272,0.467295,0.000000,0.395954,0.000000,0.311286,0.004524,0.379366


In [28]:
ev_files = [('$R$', os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior', 'case-topics-filtered-phrasestop-unigram_dir_mu_1050.00.run'), None)]
ev_files.append(('$R_{all}$', os.path.join(os.environ["HOME"], 'dirichlet_prior', 'case-topics-filtered-phrasestop-unigram_dir_mu_1050.00.run'), None))
for i, r in enumerate(runs):
    ev_files.append((names[i], os.path.join(BASE_DIR, r), None))

metrics = copy.copy(config.METRIC_NAMES)
metrics['set_P'] = 'P'
metrics['set_recall'] = 'R'
metrics['set_F_.5'] = 'F.5'

In [29]:
df = compute_stat_sig(ev_files, config.AUS_QREL_PATH, ['$R$', '$R_{all}$'], metrics, '1')

\begin{tabular}{lllllllllll}
\toprule
{} &           RR &       ERR@20 &         R@20 &        R@100 &           NDCG &          RBP &  Unjudged@20 &              P &              R &            F.5 \\
\midrule
$R$       &  0.5686$^{}$ &  0.3794$^{}$ &  0.4600$^{}$ &  0.7218$^{}$ &    0.4613$^{}$ &  0.3113$^{}$ &  0.0000$^{}$ &    0.0796$^{}$ &    0.7218$^{}$ &    0.1073$^{}$ \\
$R_{all}$ &  0.5687$^{}$ &  0.3794$^{}$ &  0.4600$^{}$ &  0.7218$^{}$ &  0.5108$^{**}$ &  0.3113$^{}$ &  0.0000$^{}$ &  0.0007$^{**}$ &  0.9888$^{**}$ &  0.0010$^{**}$ \\
all-or    &  0.5941$^{}$ &  0.3829$^{}$ &  0.4679$^{}$ &  0.6795$^{}$ &    0.4902$^{}$ &  0.3207$^{}$ &       1$^{}$ &    0.0240$^{}$ &    0.8801$^{}$ &    0.0282$^{}$ \\
all-and   &  0.5538$^{}$ &  0.3593$^{}$ &  0.4373$^{}$ &  0.6023$^{}$ &    0.4336$^{}$ &  0.2770$^{}$ &       1$^{}$ &    0.0413$^{}$ &    0.7612$^{}$ &    0.0372$^{}$ \\
or        &  0.5701$^{}$ &  0.3850$^{}$ &  0.4161$^{}$ &  0.5635$^{}$ &    0.4318$^{}$ &  0.3118$^{}$ &  

/home/danlocke/src/phd-notebooks/plotlib/plotters.py:355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[x][i] = "{:.4f}".format(df[x][i]) + "$^{" + ",".join(s[x + i]) + "}$"
/home/danlocke/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/home/danlocke/src/phd-notebooks/plotlib/plotters.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

In [30]:
df[metrics]

,recip_rank,err@20,recall_20,recall_100,ndcg,rbp@0.80,unjudged@20,set_P,set_recall,set_F_.5
$R$,0.5686$^{}$,0.3794$^{}$,0.4600$^{}$,0.7218$^{}$,0.4613$^{}$,0.3113$^{}$,0.0000$^{}$,0.0796$^{}$,0.7218$^{}$,0.1073$^{}$
$R_{all}$,0.5687$^{}$,0.3794$^{}$,0.4600$^{}$,0.7218$^{}$,0.5108$^{**}$,0.3113$^{}$,0.0000$^{}$,0.0007$^{**}$,0.9888$^{**}$,0.0010$^{**}$
all-or,0.5941$^{}$,0.3829$^{}$,0.4679$^{}$,0.6795$^{}$,0.4902$^{}$,0.3207$^{}$,1$^{}$,0.0240$^{}$,0.8801$^{}$,0.0282$^{}$
all-and,0.5538$^{}$,0.3593$^{}$,0.4373$^{}$,0.6023$^{}$,0.4336$^{}$,0.2770$^{}$,1$^{}$,0.0413$^{}$,0.7612$^{}$,0.0372$^{}$
or,0.5701$^{}$,0.3850$^{}$,0.4161$^{}$,0.5635$^{}$,0.4318$^{}$,0.3118$^{}$,3$^{}$,0.0328$^{}$,0.6466$^{}$,0.0406$^{}$
and,0.4454$^{}$,0.2701$^{}$,0.2211$^{}$,0.2588$^{}$,0.2356$^{}$,0.2193$^{}$,5$^{}$,0.1011$^{}$,0.2735$^{}$,0.0773$^{}$


In [31]:
write_table('tables/ausnl-phrase-filter', bold_max(df[metrics]).rename(columns=metrics).round(4).to_latex(escape=False))

<Figure size 1152x288 with 1 Axes>

In [17]:
inter = Interpolater(os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior', 'case-topics-filtered-phrasestop-unigram_dir_mu_1050.00.run'), normalize=True)

vals = [0]#, 300, 1050, 3000]
dfs = []

for v in vals:
    interped_dfs = []
    for _lambda in np.arange(0, 1.0, 0.01):
        inter.interpolate(os.path.join(os.environ["HOME"], 'go/src/github.com/dan-locke/phd/phrase-score/case-topics-phrase-scoring-mu-{0:.2f}.run'.format(v)), _lambda, 'tmp.run')
        interped_dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'])[0])
    dfs.append(interped_dfs)

In [18]:
to = 50
metrics = copy.copy(config.METRIC_NAMES)
del metrics['recall_100']

phrase_interp_fig = plot_tune_1d_comp(['base', 'mle', 'mu=300', 'mu=1050', 'mu=3000'], metrics, [[base_df for x in range(to+1)]] + [x[:to+1] for x in dfs], 0.00, (to)/100, 0.01, legend_x=0.965, ylims=ylims, styles=['--'])

<Figure size 1152x432 with 6 Axes>

In [19]:
# modified so does stat sig to not 0 
def select_1d_max_with_interp(display_names, dfs, start, increment, name, interp, base_qry, base_df, base_val, path, qrel_path, rel_level, metrics=None):
    measure_max = {}
    for i in range(len(display_names)):
        for j in range(1, len(dfs[i])):
            for m in dfs[i][j].index:
                if m not in metrics: 
                    continue 
                val = dfs[i][j][m]
                if (display_names[i], metrics[m]) not in measure_max: 
                    measure_max[(display_names[i], metrics[m])] = {'-': val, name: '{0:.2f}'.format(j*increment+start)}
                else: 
                    if measure_max[(display_names[i], metrics[m])]['-'] < val:
                        measure_max[(display_names[i], metrics[m])] = {'-': val, name: '{0:.2f}'.format(j*increment+start)}

    back_metric = {v: k for k, v in metrics.items()}
    for k, v in measure_max.items():
        if k[1] == 'Unjudged@20':
            continue
        _l = float(v[name])
        if _l == 0.00:
            v['-'] = '{0:.4f}'.format(v['-'])
        else:
            print(path, k[0])
            interp.interpolate(path.format(k[0]), _l, 'tmp.run')
            comp = load_dfs(qrel_path, rel_level, '', ['tmp.run'], per_query=True)[0]
            p = stats.ttest_rel(base_qry[back_metric[k[1]]], comp[back_metric[k[1]]]).pvalue
            if p < 0.01:
                v['-'] = '{0:.4f}'.format(v['-'])+'$^{**}$'
            elif p < 0.05:
                v['-'] = '{0:.4f}'.format(v['-'])+'$^{*}$'
            else:
                v['-'] = '{0:.4f}'.format(v['-'])
                
        
    for x in base_df.items():
        if x[0] not in metrics: 
            continue
        measure_max[('base', metrics[x[0]])] = {'-': '{0:.4f}'.format(x[1]), name: '{0:.2f}'.format(base_val)}
        
    max_df = pd.DataFrame.from_dict(measure_max).stack().unstack(level=0)
    return max_df.reindex(list(metrics.values()))

In [20]:
phrase_interp_max = select_1d_max_with_interp([300, 1050, 3000], dfs, 0.0, 0.01, '$\lambda$', inter, base_qry, base_df, 1050, os.path.join(os.environ["HOME"], 'go/src/github.com/dan-locke/phd/phrase-score/case-topics-phrase-scoring-mu-{0:.2f}.run'), config.AUS_QREL_PATH, config.AUS_REL_LEVEL, metrics=metrics).T

IndexError: list index out of range

In [89]:
phrase_interp_max

RR   ERR@20     R@20     NDCG           RBP Unjudged@20
300  $\lambda$     0.05     0.06     0.12     0.05          0.07        0.99
     -           0.5156   0.3367   0.4513   0.4311  0.2848$^{*}$     6.18947
1050 $\lambda$     0.05     0.06     0.11     0.06          0.06        0.99
     -           0.5162   0.3378   0.4528   0.4320        0.2865     6.06316
3000 $\lambda$     0.06     0.03     0.09     0.06          0.06        0.99
     -           0.5167   0.3408   0.4523   0.4327        0.2868     6.08421
base $\lambda$  1050.00  1050.00  1050.00  1050.00       1050.00     1050.00
     -           0.5570   0.3747   0.4706   0.4516        0.3062      2.7895